# Imports

In [1]:
%load_ext google.colab.data_table
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Data

In [2]:
movies = pd.read_csv('/content/drive/MyDrive/Datasets/Movielens 1M/movies.csv', encoding= 'unicode_escape')
movies.head()

,movie_id,movie,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
ratings = pd.read_csv('/content/drive/MyDrive/Datasets/Movielens 1M/ratings.csv')
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
df = pd.merge(ratings, movies, on='movie_id')
df.head()

,user_id,movie_id,rating,timestamp,movie,genres
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama


# Feature Engineering

In [5]:
movie_count = df.groupby('movie')['rating'].count()

In [6]:
movie_count

movie
$1,000,000 Duck (1971)                                   37
'Night Mother (1986)                                     70
'Til There Was You (1997)                                52
'burbs, The (1989)                                      303
...And Justice for All (1979)                           199
                                                       ... 
Zed & Two Noughts, A (1985)                              29
Zero Effect (1998)                                      301
Zero Kelvin (Kjü¾¦¼rlighetens kjü¾¼tere) (1995)      2
Zeus and Roxanne (1997)                                  23
eXistenZ (1999)                                         410
Name: rating, Length: 3706, dtype: int64

In [7]:
df = df.merge(movie_count, on='movie')
df.head()

,user_id,movie_id,rating_x,timestamp,movie,genres,rating_y
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,1725
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama,1725
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama,1725
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama,1725
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama,1725


In [8]:
df.columns = ['user_id', 'movie_id', 'rating', 'timestamp', 'movie', 'genres', 'number_of_ratings']

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000209 entries, 0 to 1000208
Data columns (total 7 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   user_id            1000209 non-null  int64 
 1   movie_id           1000209 non-null  int64 
 2   rating             1000209 non-null  int64 
 3   timestamp          1000209 non-null  int64 
 4   movie              1000209 non-null  object
 5   genres             1000209 non-null  object
 6   number_of_ratings  1000209 non-null  int64 
dtypes: int64(5), object(2)
memory usage: 61.0+ MB


In [10]:
Popularity_Threshold = 50
pop_movies = df[df['number_of_ratings'] > Popularity_Threshold]

In [13]:
pivot_table = pop_movies.pivot_table(index='movie', columns='user_id', values='rating').fillna(0)
pivot_table.head()

user_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,6001,6002,6003,6004,6005,6006,6007,6008,6009,6010,6011,6012,6013,6014,6015,6016,6017,6018,6019,6020,6021,6022,6023,6024,6025,6026,6027,6028,6029,6030,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movie,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
'Night Mother (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"'burbs, The (1989)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...And Justice for All (1979),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,4.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0


# Model

In [61]:
from scipy.sparse import csr_matrix
csr = csr_matrix(pivot_table.values)

In [62]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(csr)

NearestNeighbors(algorithm='brute', metric='cosine')

In [63]:
query = np.random.choice(pivot_table.shape[0])
print(query)
distances, indices = knn.kneighbors(pivot_table.iloc[querry,:].values.reshape(1,-1), n_neighbors=6)

1200


In [64]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(pivot_table.index[query]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, pivot_table.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for Jumanji (1995):

1: Robocop 2 (1990), with distance of 0.47465220270235253:
2: Judge Dredd (1995), with distance of 0.4805027274572491:
3: Stargate (1994), with distance of 0.4840053273756759:
4: Johnny Mnemonic (1995), with distance of 0.48673153708813766:
5: Lost in Space (1998), with distance of 0.5039311888517726:
